该文件用于判断
- 原始图像是否双峰分布/统计分类结果筛选不合理数据，根据面向对象统计形态、方差等

In [8]:
import ee
import geemap
import geemap.chart as chart
import numpy as np
import pandas as pd
import copy
import math
import time
import traceback
import sys,os
from Basic_tools import *
from New_Correct import *
from Correct_filter import *
from S2_filter import *
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

geemap version = 0.20.3
geemap path = ['D:\\Code_base\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


In [9]:
#--------------------------预加载冰湖数据
Glacial_lake = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt')

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_C = Glacial_lake.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_R = Glacial_lake.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
# Glacial_lake_A_GeoList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Num_list = Glacial_lake.size().getInfo()
Glacial_lake_A_GeoList = Glacial_lake.toList(Num_list)
Glacial_lake_C_CentriodList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_R_RectangleList = ee.List(Glacial_lake_C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
#--------------------确定时间范围
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
year = '2019'
START_DATE  = ee.Date(year + '-06-01')
END_DATE   = ee.Date(year + '-08-30')
S2_START_DATE = ee.Date(year + '-06-01')
S2_END_DATE  = ee.Date(year + '-09-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
#-------------------DEM选择
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)
#-------------------几何畸变参数
models = ['volume', 'surface', None]     # 地形矫正模型
Model = models[0]
Origin_scale = 10
projScale = 30
#-------------------S2 clodless参数
CLOUD_FILTER = 60         # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15        # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15       # 非水暗像素判断阈值
CLD_PRJ_DIST = 1         # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50            # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input
# ------------------分类方法和参数确定
from Extract_algorithm import Reprocess,save_parms
save_dir=os.path.join(os.getcwd(),'{}补齐结果'.format(year))
if not os.path.exists(save_dir) : os.makedirs(save_dir) ; os.chdir(save_dir)

Methods = ('Kmean','SNIC_Kmean','Xmeans','CascadeKMeans','otsu','isodata')
resultbands = (0,1)
Imgs = ('s1_unit_mean_',) # 's1_unit_max_','s1_unit_min_'
Bands = (['VV_gamma0_flatDB'],['VV_gamma0_flatDB','VH_gamma0_flatDB'])
mode = 'gpd'
savenmae = '{}SouthwestGLs'.format(year)
# ------------------重载变量参数
Reload = True
dest = 'users/mrwurenzhe/test'
metaName = 'test.csv'
eeUser = 'mrwurenzhe@furwas.com'
overwrite = 'yes'
delgeeUp = False    # 删除gee远端存储

In [ ]:
i = 100
start_time = time.time()
AOI_Centriod = ee.Feature.geometry(Glacial_lake_C_CentriodList.get(i))
AOI_GLFeature = ee.Feature(Glacial_lake_A_GeoList.get(i))
AOI_GL = AOI_GLFeature.geometry()
AOI_Rectangle = ee.Feature.geometry(Glacial_lake_R_RectangleList.get(i))
AOI_area = AOI_GLFeature.get('GL_Area_km').getInfo()
circularity = AOI_GLFeature.get('GL_Circula').getInfo()

# 缩小冰湖面积
AOI_ShrinKbuffer = AOI_GL.buffer(distance=math.log(AOI_area+1,10) * -3000 *circularity)
AOI_ShrinKbuffer_Centroid = AOI_ShrinKbuffer.centroid()

# 检测是否内缩为空，否则用AOI_area替代
if AOI_ShrinKbuffer.coordinates().getInfo() != [] :
    FilterBound = AOI_ShrinKbuffer_Centroid
else:
    FilterBound = AOI_Centriod

# 边界不可过大，否则会影响局部自适应提取效果
AOI_buffer = AOI_Rectangle.buffer(distance=math.log(AOI_area+1,5) * 1200 + 100).bounds()

# 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,Filter='RefinedLee',FilterSize=30)

# 判断是否synth
synthesis_a  = ee.Image(s1_ascending).get('synthesis').getInfo()
synthesis_d  = ee.Image(s1_descending).get('synthesis').getInfo()
if synthesis_a or synthesis_d:synthesis=1
else:synthesis=0

In [ ]:
img = img.select(band).clip(region)
img = minmax_norm(img,band,region,scale=Origin_scale,withbound=False)
img = img.reproject(proj).clip(region)